In [1]:
import pandas as pd

HS2012 nomenclature was obtained from [europa.eu](http://ec.europa.eu/eurostat/ramon/nomenclatures/index.cfm?TargetUrl=LST_CLS_DLD&StrNom=HS_2012&StrLanguageCode=EN&StrLayoutCode=HIERARCHIC). The MDB file was converted to CSV with the following command:

```
mdb-export HS_2012_MDB.mdb "HS 2012 - STRUCTURE BG CS DA DE EL EN ES FR IT PT SV" > hs2012.csv
```

In [10]:
cols = ['CNKEY', 'CN', 'PURE_HS_CODE', 'LEVEL', 'EN', 'ES']
dtype = {c: str for c in cols}
dtype['LEVEL'] = int
hs = pd.read_csv('hs2012.csv', usecols=cols, dtype=dtype)

In [56]:
level5 = hs[hs.LEVEL == 5]
level5 = level5.rename(columns={'EN': 'level5_en', 'ES': 'level5_es', 'PURE_HS_CODE': 'level5'})
level5.loc[:, 'level5'] = level5.level5.apply(lambda l: l.replace('.', ''))
level5.loc[:, 'level3'] = level5.level5.apply(lambda l: l[:4])

In [67]:
level3 = hs[hs.LEVEL == 3]
level3 = level3.rename(columns={'EN': 'level3_en', 'ES': 'level3_es', 'PURE_HS_CODE': 'level3'})
level3.loc[:, 'level3'] = level3.level3.astype(str).apply(lambda h: h.replace('.', '')[:4])
level3 = level3.merge(level5, on='level3')
level3.loc[:, 'level2'] = level3.level3.astype(str).apply(lambda h: h[:2])

In [73]:
level2 = hs[hs.LEVEL == 2]
level2 = level2.rename(columns={'EN': 'level2_en', 'ES': 'level2_es', 'PURE_HS_CODE': 'level2'})
level2 = level2.merge(level3, on='level2')
level2.loc[:, 'level2_en'] = level2.level2_en.apply(lambda h: h.split(' - ')[1].title())
level2.loc[:, 'level2_es'] = level2.level2_es.apply(lambda h: h.split(' - ')[1].title())


In [79]:
csv = level2[['level2', 'level2_en', 'level2_es', 'level3', 'level3_en', 'level3_es', 'level5', 'level5_en', 'level5_es']].reset_index()

,index,level2,level2_en,level2_es,level3,level3_en,level3_es,level5,level5_en,level5_es
0,0,01,Live Animals,Animales Vivos,0101,"Live horses, asses, mules and hinnies","Caballos, asnos, mulos y burdéganos, vivos",010121,Pure-bred breeding animals,Reproductores de raza pura
1,1,01,Live Animals,Animales Vivos,0101,"Live horses, asses, mules and hinnies","Caballos, asnos, mulos y burdéganos, vivos",010129,Other,Los demás
2,2,01,Live Animals,Animales Vivos,0102,Live bovine animals,Animales vivos de la especie bovina,010221,Pure-bred breeding animals,Reproductores de raza pura
3,3,01,Live Animals,Animales Vivos,0102,Live bovine animals,Animales vivos de la especie bovina,010229,Other,Los demás
4,4,01,Live Animals,Animales Vivos,0102,Live bovine animals,Animales vivos de la especie bovina,010231,Pure-bred breeding animals,Reproductores de raza pura
5,5,01,Live Animals,Animales Vivos,0102,Live bovine animals,Animales vivos de la especie bovina,010239,Other,Los demás
6,6,01,Live Animals,Animales Vivos,0103,Live swine,Animales vivos de la especie porcina,010391,Weighing less than 50 kg,De peso inferior a 50 kg
7,7,01,Live Animals,Animales Vivos,0103,Live swine,Animales vivos de la especie porcina,010392,Weighing 50 kg or more,De peso superior o igual a 50 kg
8,8,01,Live Animals,Animales Vivos,0105,"Live poultry, that is to say, fowls of the spe...","Gallos, gallinas, patos, gansos, pavos (gallip...",010511,Fowls of the species Gallus domesticus,Gallos y gallinas
9,9,01,Live Animals,Animales Vivos,0105,"Live poultry, that is to say, fowls of the spe...","Gallos, gallinas, patos, gansos, pavos (gallip...",010512,Turkeys,Pavos (gallipavos)
